In [ ]:
!pip install unsloth


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install wandb -qqq


In [ ]:
import wandb
wandb.login(key="YOUR_KEY")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/administrator/.netrc
wandb: Currently logged in as: vanlethai12042002 (vanlethai12042002-ton-duc-thang-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from huggingface_hub import login

login(token="YOUR_KEY")

/home/administrator/VanIT/venVan/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "thailevann/Qwen3-4B_model_CT_DVCQG_v1",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A30. Num GPUs = 2. Max memory: 23.486 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.4.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [ ]:
from unsloth.chat_templates import standardize_sharegpt
from datasets import load_dataset, Dataset, concatenate_datasets
dataset = load_dataset("thailevann/DVCQG_finetune_QA", split = "train")


In [ ]:
dataset_without_reasoning = dataset.filter(
    lambda x: (x.get("reason") or "").strip() == ""
)

In [ ]:
def convert_conversations_to_chat_format_non_reasoning(examples):
    question = examples.get("instruction", "").strip()
    answer = examples.get("output", "").strip()

    # Bỏ nếu thiếu nội dung
    if not question or not answer:
        return {"conversation": []}

    # Format prompt thân thiện
    user_prompt = f"""Bạn là một trợ lý AI hỗ trợ tra cứu dịch vụ công. Hãy trả lời câu hỏi dưới đây:
## Câu hỏi:
{question}"""

    chat_conversations = [
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": answer}
    ]

    return {"conversation": chat_conversations}


In [ ]:
for i, item in enumerate(dataset_without_reasoning):
    if (
        item is None
        or not isinstance(item, dict)
        or "instruction" not in item
        or "output" not in item
        or not isinstance(item["instruction"], str)
        or not isinstance(item["output"], str)
    ):
        print(f"❌ Lỗi tại index {i}: {item}")


In [ ]:
converted_data_non_reasoning = [convert_conversations_to_chat_format_non_reasoning(data) for data in dataset_without_reasoning]
dataset_without_reasoning = Dataset.from_list(converted_data_non_reasoning )
dataset_without_reasoning = standardize_sharegpt(dataset_without_reasoning)


In [ ]:
from datasets import Dataset

dataset_without_reasoning = dataset_without_reasoning.filter(lambda example: len(example["conversation"]) > 0)


In [ ]:
print(f"Số lượng mẫu còn lại: {len(dataset_without_reasoning)}")


In [ ]:
non_reasoning_conversations = tokenizer.apply_chat_template(
    dataset_without_reasoning["conversation"],
    tokenize = False,
)

In [ ]:
non_reasoning_conversations[1]

'<|im_start|>user\nBạn là một trợ lý AI hỗ trợ tra cứu dịch vụ công. Hãy trả lời câu hỏi dưới đây:\n## Câu hỏi:\nTra cứu danh mục các TTHC thực hiện tại Bộ phận một cửa Bộ Xây dựng bằng cách nào?<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nCác TTHC được thực hiện tại Bộ phận một cửa được công bố trong Quyết định số 1269/QĐ-BXD ngày 09/10/2018 của Bộ trưởng Bộ Xây dựng về việc công bố danh mục TTHC thực hiện tiếp nhận và trả kết quả tại BPMC của Bộ Xây dựng (được công khai trên Cổng thông tin điện tử Bộ Xây dựng tại địa chỉ www.moc.gov.vn)<|im_end|>\n'

In [ ]:
import pandas as pd
non_reasoning_subset = pd.Series(non_reasoning_conversations)


'\nnon_reasoning_subset = non_reasoning_subset.sample(\n    int(len(reasoning_conversations) * (1.0 - chat_percentage)),\n    random_state = 2407,\n)\n'

In [ ]:
import pandas as pd
data = pd.Series(non_reasoning_subset)

data.name = "text"

from datasets import Dataset
combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed = 3407)


In [ ]:
combined_dataset

Dataset({
    features: ['text'],
    num_rows: 61439
})

In [ ]:
data.head()

0    <|im_start|>user\nBạn là một trợ lý AI hỗ trợ ...
1    <|im_start|>user\nBạn là một trợ lý AI hỗ trợ ...
2    <|im_start|>user\nBạn là một trợ lý AI hỗ trợ ...
3    <|im_start|>user\nBạn là một trợ lý AI hỗ trợ ...
4    <|im_start|>user\nBạn là một trợ lý AI hỗ trợ ...
Name: text, dtype: object

In [ ]:
# Use wandb
import wandb
wandb.init(
    project="Chatbot-dvc",
    name="Qwen3-4B-CT-SFT_dvcqg_v4"
)


In [ ]:
from trl import SFTConfig, SFTTrainer


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=combined_dataset,
    eval_dataset=None,
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        warmup_steps=100,
        learning_rate = 2e-6 ,
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="./checkpoints",
        save_strategy="steps",
        save_steps=10,
        save_total_limit=2,
        #fp16=True,
        bf16 = True,
        fp16 = False,
    ),
)



Unsloth: Tokenizing ["text"] (num_proc=16): 100%|█| 61439/61439 [00:16<00:00, 36


In [ ]:
trainer_stats = trainer.train()

# Push bản cuối cùng (optional nếu bạn dùng "every_save")
model.push_to_hub("thailevann/Qwen3-4B_SFT_dvcqg_v4")
tokenizer.push_to_hub("thailevann/Qwen3-4B_SFT_dvcqg_v4")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 61,439 | Num Epochs = 1 | Total steps = 1,920
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 264,241,152/4,000,000,000 (6.61% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.541400
20,1.491200
30,1.444000
40,1.336000
50,1.270000
60,1.152900
70,1.049900
80,0.963400
90,0.967400
100,0.958300


adapter_model.safetensors: 100%|███████████| 2.61G/2.61G [02:31<00:00, 17.2MB/s]


Saved model to https://huggingface.co/thailevann/Qwen3-4B_SFT_dvcqg_v4


tokenizer.json: 100%|██████████████████████| 11.4M/11.4M [00:00<00:00, 17.3MB/s]
